<a href="https://colab.research.google.com/github/rayaneghilene/BERT_classification/blob/main/RoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of the performance RoBERTa on a classification task

Modify the following variables based on your data
* **DATAPATH**: is the path to your dataset in your environment

In [1]:
DATAPATH = '/content/Multitarget-CONAN (2).csv'

## Data  

In [2]:
# Install and import the required libraries
!pip install -q accelerate==0.21.0 --progress-bar off
!pip install -q peft==0.4.0 --progress-bar off
!pip install -q bitsandbytes==0.40.2 --progress-bar off
!pip install -q transformers==4.31.0 --progress-bar off
!pip install -q trl==0.4.7 --progress-bar off


In [3]:
!pip install nltk --progress-bar off
!pip install keras --progress-bar off
!pip install tensorflow --progress-bar off
!pip install tensorflow_hub --progress-bar off
!pip install transformers --progress-bar off

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import log_loss

## I - RoBERTa

In [5]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

df = pd.read_csv('/content/dataset.csv')

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_encodings = tokenizer(list(train_df['HATE_SPEECH']), truncation=True, padding=True, return_tensors="tf")
val_encodings = tokenizer(list(val_df['HATE_SPEECH']), truncation=True, padding=True, return_tensors="tf")

train_labels = tf.convert_to_tensor(list(train_df['label']))
val_labels = tf.convert_to_tensor(list(val_df['label']))

model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=8)

optimizer = Adam(learning_rate=5e-5)
loss_fn = SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), val_labels))

train_dataset = train_dataset.batch(16).shuffle(buffer_size=1000)
val_dataset = val_dataset.batch(64)

model.fit(train_dataset, epochs=3, validation_data=val_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification

Epoch 1/3
251/251 [==============================] - 138s 343ms/step - loss: 0.3866 - accuracy: 0.8956 - val_loss: 0.1687 - val_accuracy: 0.9620
Epoch 2/3
251/251 [==============================] - 84s 336ms/step - loss: 0.1401 - accuracy: 0.9618 - val_loss: 0.1476 - val_accuracy: 0.9670
Epoch 3/3
251/251 [==============================] - 79s 316ms/step - loss: 0.1008 - accuracy: 0.9743 - val_loss: 0.1666 - val_accuracy: 0.9630
